**Подключение библиотек и скриптов**

In [1]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import (roc_auc_score, roc_curve, auc, confusion_matrix, \
                             accuracy_score, classification_report, \
                             precision_recall_curve, recall_score)

**Пути к директориям и файлам**

In [2]:
!wget 'https://drive.google.com/uc?id=1wcpCP20G0yJpr_yVT1Hci6hNWh7FXV9-' -O training_project_data.csv

--2024-05-28 05:53:37--  https://drive.google.com/uc?id=1wcpCP20G0yJpr_yVT1Hci6hNWh7FXV9-
Resolving drive.google.com (drive.google.com)... 172.217.203.139, 172.217.203.138, 172.217.203.113, ...
Connecting to drive.google.com (drive.google.com)|172.217.203.139|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1wcpCP20G0yJpr_yVT1Hci6hNWh7FXV9- [following]
--2024-05-28 05:53:37--  https://drive.usercontent.google.com/download?id=1wcpCP20G0yJpr_yVT1Hci6hNWh7FXV9-
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.98.132, 2607:f8b0:400c:c1a::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.98.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1159496 (1.1M) [application/octet-stream]
Saving to: ‘training_project_data.csv’

training_project_da 100%[===================>]   1.11M  --.-KB/s    in 0.03s   

2024-05-28 05:53

In [3]:
DATASET_PATH = './training_project_data.csv'

### Постановка задачи<a class="anchor" id="task"></a>

Требуется на основании демографических данных о клиентах банка, истории их платежей и состоянии счета, предсказать невыполнение их обязательств по кредитной карте в следующем месяце.

**Описание датасета**

* **LIMIT_BAL** - Сумма предоставленного кредита
* **SEX** - Пол (1=мужчина, 2=женщина)
* **EDUCATION** - Образование (1=аспирантура, 2=университет, 3=старшая школа, 4=прочее, 5=неизвестно, 6=неизвестно)
* **MARRIAGE** - Семейное положение (1=женат/замужен, 2=не женат/не замужем, 3=прочее)
* **AGE** - Возраст (в годах)
* **PAY_1** - Статус погашения в Сентябре (-1=погашен полностью, 0=погашен частично, 1=отсрочка платежа на один месяц, .., 3=отсрочка платежа на три месяца и более)
* **PAY_2** - Статус погашения в Августе
* **PAY_3** - Статус погашения в Июле
* **PAY_4** - Статус погашения в Июне
* **PAY_5** - Статус погашения в Мае
* **PAY_6** - Статус погашения в Апреле
* **BILL_AMT1** - Сумма выписки по счету в Сентябре
* **BILL_AMT2** - Сумма выписки по счету в Августе
* **BILL_AMT3** - Сумма выписки по счету в Июле
* **BILL_AMT4** - Сумма выписки по счету в Июне
* **BILL_AMT5** - Сумма выписки по счету в Мае
* **BILL_AMT6** - Сумма выписки по счету в Апреле
* **PAY_AMT1** - Сумма предыдущего платежа в Сентябре
* **PAY_AMT2** - Сумма предыдущего платежа в Августе
* **PAY_AMT3** - Сумма предыдущего платежа в Июле
* **PAY_AMT4** - Сумма предыдущего платежа в Июне
* **PAY_AMT5** - Сумма предыдущего платежа в Мае
* **PAY_AMT6** - Сумма предыдущего платежа в Апреле
* **NEXT_MONTH_DEFAULT** - Просрочка платежа в следующем месяце (1=да, 0=нет)

#### Загрузка данных<a class="anchor" id="load_data"></a>

In [4]:
df = pd.read_csv(DATASET_PATH)
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,NEXT_MONTH_DEFAULT
0,150000.0,2,2,2,24,1,2,0,0,0,...,17853.0,34527.0,28277.0,0.0,2145.0,10000.0,22000.0,3000.0,5093.0,0
1,50000.0,2,3,1,46,3,3,3,3,2,...,30748.0,30162.0,30016.0,0.0,0.0,0.0,0.0,3100.0,798.0,1
2,150000.0,2,2,1,41,-1,-1,-1,-1,0,...,6784.0,6923.0,14116.0,0.0,0.0,6784.0,159.0,167.0,760.0,0
3,150000.0,2,2,2,35,0,0,0,0,0,...,55182.0,48983.0,50136.0,3500.0,2000.0,20000.0,2000.0,2000.0,2000.0,0
4,70000.0,2,1,1,35,1,2,2,2,2,...,47026.0,45971.0,47573.0,0.0,6500.0,1800.0,0.0,2500.0,2000.0,0


In [5]:
df.shape

(10000, 24)

In [6]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 24 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   LIMIT_BAL           10000 non-null  float64
 1   SEX                 10000 non-null  int64  
 2   EDUCATION           10000 non-null  int64  
 3   MARRIAGE            10000 non-null  int64  
 4   AGE                 10000 non-null  int64  
 5   PAY_1               10000 non-null  int64  
 6   PAY_2               10000 non-null  int64  
 7   PAY_3               10000 non-null  int64  
 8   PAY_4               10000 non-null  int64  
 9   PAY_5               10000 non-null  int64  
 10  PAY_6               10000 non-null  int64  
 11  BILL_AMT1           10000 non-null  float64
 12  BILL_AMT2           10000 non-null  float64
 13  BILL_AMT3           10000 non-null  float64
 14  BILL_AMT4           10000 non-null  float64
 15  BILL_AMT5           10000 non-null  float64
 16  BILL_

#### Приведение типов<a class="anchor" id="cast"></a>

In [7]:
for colname in ['SEX', 'EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5', 'PAY_6']:
    df[colname] = df[colname].astype(str)

In [8]:
df.dtypes

LIMIT_BAL             float64
SEX                    object
EDUCATION              object
MARRIAGE               object
AGE                     int64
PAY_1                  object
PAY_2                  object
PAY_3                  object
PAY_4                  object
PAY_5                  object
PAY_6                  object
BILL_AMT1             float64
BILL_AMT2             float64
BILL_AMT3             float64
BILL_AMT4             float64
BILL_AMT5             float64
BILL_AMT6             float64
PAY_AMT1              float64
PAY_AMT2              float64
PAY_AMT3              float64
PAY_AMT4              float64
PAY_AMT5              float64
PAY_AMT6              float64
NEXT_MONTH_DEFAULT      int64
dtype: object

### Обзор данных<a class="anchor" id="review"></a>

**Обзор целевой переменной**

In [9]:
df['NEXT_MONTH_DEFAULT'].value_counts()

NEXT_MONTH_DEFAULT
0    7805
1    2195
Name: count, dtype: int64

**Обзор количественных признаков**

In [10]:
df.describe()

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,NEXT_MONTH_DEFAULT
count,10000.000000,10000.00000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000
mean,168182.000000,35.39600,51490.560500,49298.972000,47292.40850,43605.665800,40347.829700,38996.784300,5511.82970,5.931217e+03,5385.050600,4585.585700,4712.134300,5055.019400,0.219500
std,130118.668771,9.19536,74906.094564,71808.952137,69398.13263,65062.788893,61044.676044,59682.999879,14748.85889,2.204220e+04,18527.041096,14066.147593,14080.847065,17359.210922,0.413929
min,10000.000000,21.00000,-11545.000000,-24704.000000,-157264.00000,-65167.000000,-23003.000000,-339603.000000,0.00000,0.000000e+00,0.000000,0.000000,0.000000,0.000000,0.000000
25%,50000.000000,28.00000,3388.000000,3016.500000,2561.75000,2400.000000,1703.250000,1300.750000,1000.00000,7.492500e+02,390.000000,277.250000,300.000000,150.750000,0.000000
50%,140000.000000,34.00000,22896.500000,21576.000000,20370.50000,19194.000000,18261.000000,17338.000000,2163.50000,2.001000e+03,1800.000000,1500.000000,1543.000000,1500.000000,0.000000
75%,240000.000000,41.00000,67834.750000,64243.500000,60783.25000,55731.250000,50491.000000,49696.000000,5001.00000,5.000000e+03,4651.500000,4014.250000,4100.000000,4000.000000,0.000000
max,1000000.000000,75.00000,964511.000000,983931.000000,689643.00000,891586.000000,927171.000000,961664.000000,493358.00000,1.227082e+06,896040.000000,432130.000000,417990.000000,422000.000000,1.000000


**Обзор номинативных признаков**

* **SEX** - Пол (1=мужчина, 2=женщина)
* **EDUCATION** - Образование (1=аспирантура, 2=университет, 3=старшая школа, 4=прочее, 5=неизвестно, 6=неизвестно)
* **MARRIAGE** - Семейное положение (1=женат/замужен, 2=не женат/не замужем, 3=прочее)

In [11]:
for cat_colname in df.select_dtypes(include='object').columns:
    print('\n\n' + str(df[cat_colname].value_counts()) + '\n' + '*' * 100 + '\n')



SEX
2    6068
1    3932
Name: count, dtype: int64
****************************************************************************************************



EDUCATION
2    4678
1    3496
3    1674
5      90
4      42
6      18
0       2
Name: count, dtype: int64
****************************************************************************************************



MARRIAGE
2    5332
1    4547
3     108
0      13
Name: count, dtype: int64
****************************************************************************************************



PAY_1
0     4895
-1    2852
1     1217
2      881
3      155
Name: count, dtype: int64
****************************************************************************************************



PAY_2
0     5178
-1    3318
2     1318
3      175
1       11
Name: count, dtype: int64
****************************************************************************************************



PAY_3
0     5200
-1    3379
2     1288
3      131
1        2
Name: count,

#### Обработка пропусков<a class="anchor" id="gaps"></a>

In [12]:
df.isna().sum()

LIMIT_BAL             0
SEX                   0
EDUCATION             0
MARRIAGE              0
AGE                   0
PAY_1                 0
PAY_2                 0
PAY_3                 0
PAY_4                 0
PAY_5                 0
PAY_6                 0
BILL_AMT1             0
BILL_AMT2             0
BILL_AMT3             0
BILL_AMT4             0
BILL_AMT5             0
BILL_AMT6             0
PAY_AMT1              0
PAY_AMT2              0
PAY_AMT3              0
PAY_AMT4              0
PAY_AMT5              0
PAY_AMT6              0
NEXT_MONTH_DEFAULT    0
dtype: int64

#### Обработка выбросов<a class="anchor" id="outliers"></a>

**EDUCATION**

In [13]:
df.loc[df['EDUCATION'] == '0', 'EDUCATION'] = df['EDUCATION'].mode()[0]

**MARRIAGE**

In [14]:
df.loc[df['MARRIAGE'] == '0', 'MARRIAGE'] = df['MARRIAGE'].mode()[0]

#### Построение новых признаков<a class="anchor" id="new_features"></a>

**Dummies**

In [16]:
df['IS_MALE'] = df['SEX'].map({'1':'1', '2':'0'}).astype(int)
df.head()

,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,NEXT_MONTH_DEFAULT,IS_MALE
0,150000.0,2,2,2,24,1,2,0,0,0,...,34527.0,28277.0,0.0,2145.0,10000.0,22000.0,3000.0,5093.0,0,0
1,50000.0,2,3,1,46,3,3,3,3,2,...,30162.0,30016.0,0.0,0.0,0.0,0.0,3100.0,798.0,1,0
2,150000.0,2,2,1,41,-1,-1,-1,-1,0,...,6923.0,14116.0,0.0,0.0,6784.0,159.0,167.0,760.0,0,0
3,150000.0,2,2,2,35,0,0,0,0,0,...,48983.0,50136.0,3500.0,2000.0,20000.0,2000.0,2000.0,2000.0,0,0
4,70000.0,2,1,1,35,1,2,2,2,2,...,45971.0,47573.0,0.0,6500.0,1800.0,0.0,2500.0,2000.0,0,0


In [17]:
df.drop(columns=['SEX'], inplace=True)
df.head()

,LIMIT_BAL,EDUCATION,MARRIAGE,AGE,PAY_1,PAY_2,PAY_3,PAY_4,PAY_5,PAY_6,...,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,NEXT_MONTH_DEFAULT,IS_MALE
0,150000.0,2,2,24,1,2,0,0,0,0,...,34527.0,28277.0,0.0,2145.0,10000.0,22000.0,3000.0,5093.0,0,0
1,50000.0,3,1,46,3,3,3,3,2,0,...,30162.0,30016.0,0.0,0.0,0.0,0.0,3100.0,798.0,1,0
2,150000.0,2,1,41,-1,-1,-1,-1,0,0,...,6923.0,14116.0,0.0,0.0,6784.0,159.0,167.0,760.0,0,0
3,150000.0,2,2,35,0,0,0,0,0,0,...,48983.0,50136.0,3500.0,2000.0,20000.0,2000.0,2000.0,2000.0,0,0
4,70000.0,1,1,35,1,2,2,2,2,0,...,45971.0,47573.0,0.0,6500.0,1800.0,0.0,2500.0,2000.0,0,0


In [18]:
cat_columns = df.select_dtypes(include='object').columns
cat_columns

Index(['EDUCATION', 'MARRIAGE', 'PAY_1', 'PAY_2', 'PAY_3', 'PAY_4', 'PAY_5',
       'PAY_6'],
      dtype='object')

In [19]:
df = pd.get_dummies(df, columns=cat_columns, dtype='int')
df.head()

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,...,PAY_4_2,PAY_4_3,PAY_5_-1,PAY_5_0,PAY_5_2,PAY_5_3,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3
0,150000.0,24,65542.0,28311.0,23978.0,17853.0,34527.0,28277.0,0.0,2145.0,...,0,0,0,1,0,0,0,1,0,0
1,50000.0,46,33100.0,32319.0,31499.0,30748.0,30162.0,30016.0,0.0,0.0,...,0,1,0,0,1,0,0,1,0,0
2,150000.0,41,1650.0,0.0,0.0,6784.0,6923.0,14116.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
3,150000.0,35,97268.0,71876.0,50043.0,55182.0,48983.0,50136.0,3500.0,2000.0,...,0,0,0,1,0,0,0,1,0,0
4,70000.0,35,41730.0,40764.0,46281.0,47026.0,45971.0,47573.0,0.0,6500.0,...,1,0,0,0,1,0,0,1,0,0


### Обучение модели<a class="anchor" id="training"></a>

In [20]:
df.columns

Index(['LIMIT_BAL', 'AGE', 'BILL_AMT1', 'BILL_AMT2', 'BILL_AMT3', 'BILL_AMT4',
       'BILL_AMT5', 'BILL_AMT6', 'PAY_AMT1', 'PAY_AMT2', 'PAY_AMT3',
       'PAY_AMT4', 'PAY_AMT5', 'PAY_AMT6', 'NEXT_MONTH_DEFAULT', 'IS_MALE',
       'EDUCATION_1', 'EDUCATION_2', 'EDUCATION_3', 'EDUCATION_4',
       'EDUCATION_5', 'EDUCATION_6', 'MARRIAGE_1', 'MARRIAGE_2', 'MARRIAGE_3',
       'PAY_1_-1', 'PAY_1_0', 'PAY_1_1', 'PAY_1_2', 'PAY_1_3', 'PAY_2_-1',
       'PAY_2_0', 'PAY_2_1', 'PAY_2_2', 'PAY_2_3', 'PAY_3_-1', 'PAY_3_0',
       'PAY_3_1', 'PAY_3_2', 'PAY_3_3', 'PAY_4_-1', 'PAY_4_0', 'PAY_4_2',
       'PAY_4_3', 'PAY_5_-1', 'PAY_5_0', 'PAY_5_2', 'PAY_5_3', 'PAY_6_-1',
       'PAY_6_0', 'PAY_6_2', 'PAY_6_3'],
      dtype='object')

In [21]:
X = df.drop(columns=['NEXT_MONTH_DEFAULT'])
y = df['NEXT_MONTH_DEFAULT']

In [22]:
X.head()

,LIMIT_BAL,AGE,BILL_AMT1,BILL_AMT2,BILL_AMT3,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,...,PAY_4_2,PAY_4_3,PAY_5_-1,PAY_5_0,PAY_5_2,PAY_5_3,PAY_6_-1,PAY_6_0,PAY_6_2,PAY_6_3
0,150000.0,24,65542.0,28311.0,23978.0,17853.0,34527.0,28277.0,0.0,2145.0,...,0,0,0,1,0,0,0,1,0,0
1,50000.0,46,33100.0,32319.0,31499.0,30748.0,30162.0,30016.0,0.0,0.0,...,0,1,0,0,1,0,0,1,0,0
2,150000.0,41,1650.0,0.0,0.0,6784.0,6923.0,14116.0,0.0,0.0,...,0,0,0,1,0,0,0,1,0,0
3,150000.0,35,97268.0,71876.0,50043.0,55182.0,48983.0,50136.0,3500.0,2000.0,...,0,0,0,1,0,0,0,1,0,0
4,70000.0,35,41730.0,40764.0,46281.0,47026.0,45971.0,47573.0,0.0,6500.0,...,1,0,0,0,1,0,0,1,0,0


In [23]:
X_train, X_test, y_train, y_test = train_test_split(X, y,
                                                    test_size=0.2,
                                                    shuffle=True,
                                                    random_state=2,
                                                    stratify=y
                                                    )
X_train.shape, X_test.shape
X_train.index
y_train.value_counts(normalize=True)

NEXT_MONTH_DEFAULT
0    0.7805
1    0.2195
Name: proportion, dtype: float64

In [24]:
y_test.value_counts(normalize=True)

NEXT_MONTH_DEFAULT
0    0.7805
1    0.2195
Name: proportion, dtype: float64

In [25]:
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
X_train[X_train.columns] = scaler.fit_transform(X_train[X_train.columns])
X_test[X_train.columns] = scaler.transform(X_test[X_train.columns])

In [26]:
model = KNeighborsClassifier().fit(X_train, y_train)

In [27]:
pred_train = model.predict(X_train)
pred_test = model.predict(X_test)

In [28]:
pred_test

array([0, 0, 1, ..., 0, 1, 1])

In [29]:
y_test[:5]

7203    1
3859    0
5995    0
6470    0
9541    1
Name: NEXT_MONTH_DEFAULT, dtype: int64

### Подсчет метрик <a class="anchor" id="count_metrics"></a>

In [30]:
from sklearn.metrics import accuracy_score


accuracy_train = accuracy_score(y_train, pred_train)
accuracy_test = accuracy_score(y_test, pred_test)

print(f'Accuracy на трейне {accuracy_train}')
print(f'Accuracy на тесте {accuracy_test}')

Accuracy на трейне 0.84525
Accuracy на тесте 0.7975


In [31]:
model.score(X_train, y_train)

0.84525

In [32]:
model.score(X_test, y_test)

0.7975

In [33]:
predicted_classes = model.predict(X_test)
predicted_classes

array([0, 0, 1, ..., 0, 1, 1])

### 🧠 Упражнение: подсчет метрик вручную


#### 1 задание

Посчитайте значения TP, FP, TN, FN через цикл


In [ ]:
#@title Решение
tp = 0 # True positive
fp = 0 # False positive
fn = 0 # False negative
tn = 0 # True negative


for predicted, actual in zip(predicted_classes, y_test):

    if predicted == 1:
        if actual == 1:
            tp += 1
        else:
            fp += 1

    else:
        if actual == 1:
            fn += 1
        else:
            tn += 1

tp, fp, fn, tn

(140, 106, 299, 1455)

#### 2 задание

Посчитайте значение метрики accuracy

In [ ]:
#@title Решение
(tp + tn) / (tp + fp + fn + tn)

0.7975

#### 3 задание

Посчитайте значение метрики precision

In [ ]:
#@title Решение
precision = tp / (tp + fp)
precision

0.5691056910569106

#### 4 задание

Посчитайте значение метрики recall

In [ ]:
#@title Решение
recall = tp / (tp + fn)
recall

0.31890660592255127

### Метрики из sklearn

In [34]:
from sklearn.metrics import recall_score, precision_score

In [35]:
pred = model.predict(X_test)
pred

array([0, 0, 1, ..., 0, 1, 1])

In [36]:
recall_score(y_test, pred)

0.31890660592255127

In [37]:
precision_score(y_test, pred)

0.5691056910569106

Thus in binary classification, the count of true negatives is
C_{0,0}, false negatives is C_{1,0}, true positives is
C_{1,1} and false positives is C_{0,1}.

In [38]:
from sklearn.metrics import confusion_matrix

print(confusion_matrix(y_test, pred_test))

[[1455  106]
 [ 299  140]]


In [39]:
from sklearn.metrics import classification_report

print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88      1561
           1       0.57      0.32      0.41       439

    accuracy                           0.80      2000
   macro avg       0.70      0.63      0.64      2000
weighted avg       0.77      0.80      0.77      2000



__________
### Micro, Macro, Weighted

In [40]:
from sklearn.metrics import f1_score, precision_score, recall_score, \
                            classification_report, confusion_matrix, accuracy_score, f1_score

In [41]:
true = [0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
pred = [0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 1, 1, 0, 0]

In [42]:
confusion_matrix(true, pred)

array([[4, 1],
       [5, 5]])

**MICRO**

In [43]:
precision = 9 / 15
precision, accuracy_score(true, pred)

(0.6, 0.6)

In [44]:
recall = 9 / 15
recall, accuracy_score(true, pred)

(0.6, 0.6)

In [45]:
2 * (precision * recall) / (precision + recall)

0.6

In [46]:
recall_score(true, pred, average='micro')

0.6

In [47]:
f1_score(true, pred, average='micro')

0.6

**MACRO**

In [48]:
confusion_matrix(true, pred)

array([[4, 1],
       [5, 5]])

In [49]:
precision_0 = 4 / (4 + 5)
precision_1 = 5 / (5 + 1)
macro_pr = (precision_1 + precision_0) / 2
macro_pr

0.6388888888888888

In [50]:
precision_score(true, pred, average='macro')

0.6388888888888888

In [51]:
recall_1 = 5 / (5 + 5)
recall_0 = 4 / (4 + 1)
macro_rec = (recall_1 + recall_0) / 2
macro_rec

0.65

In [52]:
recall_score(true, pred, average='macro')

0.65

In [53]:
# 1 способ
f_score_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0)
f_score_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1)
f_score = (f_score_0 + f_score_1) / 2
f_score_0, f_score_1, f_score

(0.5714285714285714, 0.625, 0.5982142857142857)

In [54]:
# 2 способ
2 * (macro_rec * macro_pr) / (macro_pr + macro_rec)

0.6443965517241379

In [55]:
f1_score(true, pred, average='macro')

0.5982142857142857

**WEIGHTED**

In [56]:
zero = true.count(0)
one = true.count(1)
size = len(true)

In [57]:
zero / size * precision_0 + one / size * precision_1

0.7037037037037037

In [58]:
precision_score(true, pred, average='weighted')

0.7037037037037038

In [59]:
zero / size * recall_0 + one / size * recall_1

0.6

In [60]:
recall_score(true, pred, average='weighted')

0.6

In [61]:
f_score_0 = 2 * (precision_0 * recall_0) / (precision_0 + recall_0)
f_score_1 = 2 * (precision_1 * recall_1) / (precision_1 + recall_1)
zero / size * f_score_0  + one / size * f_score_1

0.6071428571428571

In [62]:
f1_score(true, pred, average='weighted')

0.6071428571428572

In [63]:
print(classification_report(y_test, pred_test))

              precision    recall  f1-score   support

           0       0.83      0.93      0.88      1561
           1       0.57      0.32      0.41       439

    accuracy                           0.80      2000
   macro avg       0.70      0.63      0.64      2000
weighted avg       0.77      0.80      0.77      2000



### Кросс-валидация

<img src='https://scikit-learn.org/stable/_images/grid_search_cross_validation.png' width=600>

In [64]:
from sklearn.model_selection import cross_val_score


model1 = KNeighborsClassifier()
cross_val_score(model1, X_train, y_train, cv=5, scoring='f1').mean()

0.411651730309379

In [65]:
model2 = KNeighborsClassifier(n_neighbors=10)
cross_val_score(model2, X_train, y_train, cv=5, scoring='f1').mean()

0.40188819215322613

In [66]:
model3 = KNeighborsClassifier(weights='distance')
cross_val_score(model3, X_train, y_train, cv=5, scoring='f1').mean()

0.40323729710195594

In [67]:
from sklearn.model_selection import cross_validate

cv_res = cross_validate(
    model1,
    X_train, y_train,
    cv=5,
    scoring=['accuracy', 'recall', 'f1']
)
cv_res

{'fit_time': array([0.01126885, 0.00896144, 0.00901747, 0.01096082, 0.00929761]),
 'score_time': array([0.16224766, 0.19346118, 0.17192912, 0.16049075, 0.15928364]),
 'test_accuracy': array([0.80375 , 0.811875, 0.79375 , 0.7875  , 0.795   ]),
 'test_recall': array([0.33903134, 0.33048433, 0.32193732, 0.31623932, 0.29829545]),
 'test_f1': array([0.43115942, 0.43527205, 0.40647482, 0.39501779, 0.39033457])}

In [68]:
cv_res['test_accuracy'].mean()

0.798375

In [69]:
cv_res = cross_validate(
    model2,
    X_train, y_train,
    cv=5,
    scoring=['accuracy', 'recall', 'f1']
)
cv_res

{'fit_time': array([0.01586437, 0.00887322, 0.00964451, 0.00861025, 0.00884247]),
 'score_time': array([0.17058778, 0.15821671, 0.17742801, 0.18595767, 0.17081356]),
 'test_accuracy': array([0.815   , 0.815   , 0.81125 , 0.815625, 0.8025  ]),
 'test_recall': array([0.28490028, 0.3019943 , 0.2962963 , 0.3019943 , 0.25568182]),
 'test_f1': array([0.40322581, 0.41732283, 0.40784314, 0.41814596, 0.36290323])}

In [70]:
cv_res['test_accuracy'].mean()

0.811875

## Дополнительные ссылки
1. Виды машинных обучений. Обучение с учителем, без и с подкреплением https://youtu.be/ku2oPMPht2I
2. Устройство проекта в машинном обучении https://youtu.be/LXc6gjip_oM
3. Метрики для задачи классификации https://www.youtube.com/playlist?list=PLkJJmZ1EJno6CLyvD4DHc32Fed6cPBHje
4. Недообучение и переобучение https://youtu.be/m7P7bnLxIc4
5. Метрики для регрессии https://youtu.be/vh2smjQyhp8
